In [1]:
import pandas as pd

### A partir d'un dataFrame cette methode va récuperer toute les cartes joués et en faire des phrases qui seront en suite passé à l'algorithme word2vec

### A games 
### On prend la colonne games

### A partir de card_history
### on recupere l'historique des cartes sorties pour chaque partie On filtre si le champs card_history existe ou si il est vide

### A partir de sentances
### Pour chaque partie on rassemble toutes les cartes sorties par "me" et "opponent", puis on recupère le nom des cartes pour crée des phrases.


In [8]:
def createsentances(data):
    games = data["games"]
    cards = [e["card_history"] for e in games if e.has_key("card_history") and e["card_history"]!=[]]
    sentances = []
    for c in cards:
        sentances.append([e["card"]["name"]for e in c if e["player"] == "me"])
        sentances.append([e["card"]["name"]for e in c if e["player"] == "opponent"])
    return sentances

creation de phrase a partir de plusieurs fichier json
listfile est une liste de fichier avec leur chemin

In [11]:
def createsentancesfiles(listfile):
    sentances = []
    for l in listfile:
        data=pd.read_json(l)
        sentances += createsentances(data)
        del data
    return sentances

In [4]:
directoryHearthstoneGames = "HeathstoneGames/"
from os import listdir
from os.path import isfile, join
filesHearthstoneGames = [directoryHearthstoneGames+f for f in listdir(directoryHearthstoneGames) if isfile(join(directoryHearthstoneGames, f))]


In [12]:
sentances = createsentancesfiles(filesHearthstoneGames)

In [6]:
data = pd.read_json("2018-01.zip");

In [9]:
sentances = createsentances(data)


# Je prend l'url avec collectible car je ne veux que les cartes disponibles pour faire un deck

In [14]:
import requests

urlApi = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"

ApiHearthStone = requests.get(urlApi).json()

In [15]:
listNameCard = [e["name"] for e in ApiHearthStone]

on filtre les cartes des parties à seulement celle disponible pour la construction de deck

In [16]:
sentances = [filter(lambda x: x in listNameCard, e) for e in sentances]

# On va utilisier gensim et plus particulièrement l'algo Word2vec

In [17]:
from gensim.models import Word2Vec

max_count exclue les mots dont leur nombre d'apparition est inférieure à celui indiqué
size nombre de noeuds
workers paralléliser sur un nombres de coeurs
ier nombre d'epochs

In [29]:
model = Word2Vec(sentances,min_count=10,size=200,workers=4,iter=100)

sauvegarde du model

In [31]:
model.save("word2vecModel")

charger le modele entrainer et de continuer l'entrainement

In [3]:
model = Word2Vec.load("word2vecModel")

In [32]:
model.wv.most_similar(positive=["Open the Waygate","Fireball"],topn=5)

[(u'Frostbolt', 0.7010962963104248),
 (u'Blizzard', 0.6939037442207336),
 (u'Frost Nova', 0.6825650930404663),
 (u'Ice Block', 0.6699504852294922),
 (u'Arcane Intellect', 0.6649154424667358)]

In [66]:
heroes = [e["name"] for e in ApiHeartStone if e["type"]=="HERO"]
filter(lambda x: x in heroes, model.wv.vocab.keys())

[u'Scourgelord Garrosh',
 u'Frost Lich Jaina',
 u'Deathstalker Rexxar',
 u'Shadowreaper Anduin',
 u'Malfurion the Pestilent',
 u'Uther of the Ebon Blade',
 u"Bloodreaver Gul'dan",
 u'Valeera the Hollow',
 u'Thrall, Deathseer']

In [64]:
model.wv.vocab.keys()

list

proposition de cartes si deck 30 ou plus aucune proposition

verifier deck depart un ou moins hero et proposition eliminer autre proposer pas besoin car present dans le dictionnaire
negative si cardClass différent 
meme classes de carte que le heros sauf neutral

In [ ]:
def proposeCards(uncompletedDeck,model,apiHS):
    cardClass = ""
    if len(uncompletedDeck) >= 30:
        return
    for e in uncompletedDeck:
       [i["cardClass"] for i in apiHS i["name"]==e]
       
    
    model.wv.most_similar(uncompletedDeck,topn=5)